<a href="https://colab.research.google.com/github/sergioGarcia91/GAIRA_UIS_2025/blob/main/02_GAIRA_Reentrenamiento_YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción

En este Notebook se describe el flujo de trabajo para:

1. **Entrenar** un modelo [YOLOv8](docs.ultralytics.com/models/yolov8/) desde cero.  
2. **Reentrenar** un modelo existente con nuevos datos.  
3. **Reanudar** un entrenamiento interrumpido (checkpoint restart).  

Para profundizar en los conceptos, parámetros y ejemplos de uso de YOLOv8, consulta la documentación oficial de [Ultralytics](https://docs.ultralytics.com/quickstart/).  


# Inicio

## Instalación de la librería

Google Colab incluye muchas librerías por defecto, pero `ultralytics` no está preinstalada. Para añadirla al entorno, ejecuta esta celda:

```bash
!pip install ultralytics


In [ ]:
# Instalar ultralytics
!pip install ultralytics

## Conexión con Google Drive

In [ ]:
# Para acceder a archivos almacenados en tu unidad de Google Drive desde este Notebook
from google.colab import drive

# Monta tu Google Drive en el directorio /content/drive
drive.mount('/content/drive')

In [ ]:
## Carga de librerías
import os
import shutil
import random
import ultralytics

In [ ]:
# Verifica la instalación y disponibilidad de GPU
ultralytics.checks()

Google Colab ofrece acceso a diversas GPUs, esenciales para acelerar el procesamiento de modelos YOLO. Cada opción presenta características distintas y una tasa de uso aproximada por hora. Esta tasa se mide en unidades de procesamiento: **100 unidades equivalen a 50 000 COP**. Colab incluye **100 unidades gratuitas al mes**; una vez agotadas, es posible adquirir paquetes de recarga. A la fecha de realización de este Notebook, sus especificaciones son:

<div align="center">

| Nombre GPU | RAM | RAM GPU | Disco     | Tasa aprox. de uso por hora (unidades) |
|:----------:|:---------:|:--------:|:---------:|:--------------------------------------:|
| **A100**   | 83.5 GB   | 40 GB    | 112.6 GB  | 6.86                                   |
| **L4**     | 53 GB     | 22.5 GB  | 112.6 GB  | 2.09                                   |
| **T4**     | 12.7 GB   | 15 GB    | 112.6 GB  | 1.43                                   |

</div>




# Entrenamiento de un modelo YOLOv8

Antes de entrenar, elige entre crear un modelo desde la configuración YAML, cargar uno preentrenado o transferir pesos personalizados:

```python
import ultralytics

# 1. Crear un modelo nuevo a partir del archivo YAML (estructura de la red)
model = ultralytics.YOLO('yolov8n-seg.yaml')

# 2. Cargar un modelo preentrenado (punto de partida recomendado)
model = ultralytics.YOLO('yolov8n-seg.pt')

# 3. Crear desde YAML y cargar pesos personalizados
pesos_personalizados = "/content/drive/MyDrive/pesos/pesos_rocas_mod.pt"
model = ultralytics.YOLO('yolov8n-seg.yaml').load(pesos_personalizados)


## Entrenamiento inicial con modelo preentrenado YOLO

Si es la primera vez que entrenas un modelo, comienza cargando un modelo preentrenado proporcionado por Ultralytics. Por ejemplo, para segmentación:

```python
# Carga del modelo de segmentación preentrenado de Ultralytics
model = ultralytics.YOLO('yolov8n-seg.pt')
```

> **Observación:** Ultralytics ofrece varias variantes de modelos YOLOv8 (p. ej., `yolov8n-seg.pt`, `yolov8x-seg.pt`, etc.) que difieren en arquitectura y tamaño de parámetro. Revisa la documentación oficial para elegir la versión más adecuada a tu proyecto.

In [ ]:
# Carga del modelo de segmentación preentrenado de Ultralytics
model = ultralytics.YOLO('yolov8n-seg.pt')

Una vez cargado el modelo, define la ruta al archivo `dataset.yaml` (generado tras la conversión con `labelme2yolov8`) e iniciar el entrenamiento mediante `model.train()`. A continuación se muestran los parámetros con sus valores por defecto:

```python
# Ruta al archivo dataset.yaml con rutas de train/val y clases
path_yaml = "/content/drive/MyDrive/ruta/a/YOLOv8Dataset/dataset.yaml"

# Inicio del entrenamiento con parámetros por defecto
results = model.train(
    data=path_yaml,
    epochs=100,
    imgsz=640,
    batch=16,
    save=True,
    save_period=-1,
    close_mosaic=10,
    lr0=0.01,
    lrf=0.01,
    optimizer='auto',
    project=None,
    resume=False,
    patience=100,
    seed=0,
)



## Descripción de parámetros

- **data**: Ruta al archivo YAML que contiene las rutas de entrenamiento (`train`), validación (`val`) y la lista de clases.  
- **epochs**: Número de épocas de entrenamiento.  
- **imgsz**: Tamaño (ancho y alto) al que redimensionar las imágenes de entrada.  
- **batch**: Tamaño de cada lote de datos.  
- **save**: Guarda checkpoints y el modelo final.  
- **save_period**: Frecuencia (en épocas) para guardar checkpoints; `-1` desactiva guardado intermedio.  
- **close_mosaic**: Número de épocas finales sin aplicar la técnica mosaic.  
- **lr0**: Learning rate inicial.  
- **lrf**: Fracción del learning rate inicial al final del entrenamiento, learning rate final = lr0*lrf.  
- **optimizer**: Optimizador a utilizar (`'auto'` selecciona automáticamente entre SGD, AdamW, etc.).  
- **project**: Directorio raíz donde se guardan los resultados; si es `None`, se crea automáticamente en `runs/`.  
- **resume**: Si es `True`, reanuda desde el último checkpoint; si es `False`, inicia un entrenamiento nuevo.  
- **patience**: Número de épocas sin mejora en la métrica para detener el entrenamiento anticipadamente.  
- **seed**: Semilla para la reproducibilidad.


In [ ]:
# Ruta al archivo dataset.yaml con rutas de train/val y clases
path_yaml = "/content/drive/MyDrive/ruta/a/YOLOv8Dataset/dataset.yaml"

# Inicio del entrenamiento con parámetros por defecto
results = model.train(
                      data=path_yaml,
                      epochs=100,
                      imgsz=640,
                      batch=16,
                      save=True,
                      save_period=-1,
                      close_mosaic=10,
                      lr0=0.01,
                      lrf=0.01,
                      optimizer='auto',
                      project=None,
                      name=None,
                      resume=False,
                      patience=100,
                      seed=0,
                  )

# Reentrenamiento del modelo `rocas_kkuuku-n`

Una vez que se ha entrenado un modelo con un objetivo específico, por ejemplo, la clasificación de rocas ígneas, sedimentarias o metamórficas, se puede realizar el entrenamiento para mejorar su desempeño con nuevos datos. En este ejemplo:

1. Cargamos el modelo y sus pesos preentrenados (`rocas_kkuuku-n`).  
2. Ajustamos los parámetros de entrenamiento según los requisitos de nuestro proyecto.  
3. Iniciamos el proceso de reentrenamiento con `model.train()`.  



In [ ]:
# Ruta al archivo de pesos del modelo a reentrenar
pesos_modelo = 'rocas_kkuuku-n.pt'
model = ultralytics.YOLO(pesos_modelo)

# Ruta al archivo dataset.yaml generado por labelme2yolov8
path_yaml = '/content/drive/MyDrive/ruta/a/YOLOv8Dataset/dataset.yaml'

# Configuración y ejecución del reentrenamiento
results = model.train(
    data=path_yaml,           # Archivo YAML con rutas de train/val y lista de clases
    epochs=200,               # Número de épocas (p.ej., 10, 100, 200, 500)
    imgsz=[1536, 1280],       # Tamaño de las imágenes (p.ej., 640, 1280, [1536,1280], [3072,2560])
    batch=16,                 # Tamaño de lote (p.ej., 32, 16, 8, 2)
    save=True,                # Guardar checkpoints y pesos finales
    save_period=5,            # Frecuencia de guardado en épocas (p.ej., 1, 5, 10)
    close_mosaic=10,          # Épocas finales sin mosaic (0 deshabilita)
    lr0=0.005,                # Learning rate inicial (p.ej., 0.01, 0.001, 0.005)
    lrf=0.01,                 # Fracción del lr0 para el lr final
    optimizer='AdamW',        # Optimizador (p.ej., 'SGD', 'AdamW', 'auto')
    project='/content/drive/MyDrive/ruta/a/guardar/Runs',  # Directorio raíz de resultados
    resume=False,             # False: nuevo entrenamiento; True: reanudar checkpoint
    seed=2,                   # Semilla para reproducibilidad
    patience=100,             # Número de épocas sin mejora en la métrica
)

# Reanudar un entrenamiento interrumpido usando resume

En Google Colab, el entrenamiento puede detenerse por límites de sesión, cortes de energía o cierres del navegador. Para continuar desde el último punto guardado, sigue estos pasos:

1. **Identifica el checkpoint**: puede ser `last.pt` (última época) o un archivo específico como `epoch15.pt`.  
2. **Carga el modelo** con ese checkpoint.  
3. **Llama a `model.train()`** con `resume=True` y sin volver a especificar los parámetros de entrenamiento (a menos que desees modificarlos).

```python
import ultralytics

# 1. Ruta al checkpoint que deseas reanudar
checkpoint = "/content/drive/MyDrive/ruta/a/RunsResultados/train/weights/last.pt"

# 2. Carga el modelo con los pesos del checkpoint
model = ultralytics.YOLO(checkpoint)

# 3. Reanuda el entrenamiento
results = model.train(resume=True)


In [ ]:
# Ruta al checkpoint para reanudar el entrenamiento
checkpoint_path = "/content/drive/MyDrive/ruta/a/RunsResultados/train/weights/epoch15.pt"

# Carga del modelo con los pesos del dicho checkpoint
model = ultralytics.YOLO(checkpoint_path)

# Reanuda el entrenamiento desde el checkpoint
results = model.train(resume=True)

# Fin